# **Diabetes Classification:**
### Base Learners: KNN, SVM

### Meta Learner: Random Forest

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
# load the daibetes dataset
df = pd.read_csv('diabetes.csv')
df.head()

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

In [53]:
features=df.drop('Outcome',axis=1)
target=df['Outcome']

In [54]:
# split into training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [55]:
# Train Base models and get out of fold prediction for all data points in training data (using CV)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

KNN_base=KNeighborsClassifier()
SVM_base=SVC()

In [56]:
# train the models using CV and get the out of fold predictions
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

KNN_oof_preds = np.zeros(len(x_train))
SVM_oof_preds = np.zeros(len(x_train))

for train_index, val_index in kf.split(x_train):
    x_tr, x_val = x_train.iloc[train_index, :], x_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    KNN_base.fit(x_tr, y_tr)
    SVM_base.fit(x_tr, y_tr)

    # these are the input features for our random forest model
    KNN_oof_preds[val_index] = KNN_base.predict(x_val)
    SVM_oof_preds[val_index] = SVM_base.predict(x_val)


In [57]:
print(x_train.shape)
print(KNN_oof_preds.shape)
print(SVM_oof_preds.shape) # same

(614, 8)
(614,)
(614,)


In [58]:
# meta model input features
arr=np.column_stack((KNN_oof_preds, SVM_oof_preds))
print(arr.shape)

(614, 2)


In [59]:
meta_train=pd.DataFrame(arr,columns=['KNN','SVM'])
meta_train.head()

KNN  SVM
0  0.0  0.0
1  1.0  0.0
2  0.0  0.0
3  0.0  1.0
4  0.0  0.0

In [60]:
# Train the meta learner
from sklearn.ensemble import RandomForestClassifier
meta_learner = RandomForestClassifier()
meta_learner.fit(meta_train, y_train) # meta_train is the Oof predictions and y train is the actual output

RandomForestClassifier()

In [61]:
# to test the meta model, first we need to get the meta_test from base models
# so, train the base models on entire training data set

KNN_base.fit(x_train, y_train)
SVM_base.fit(x_train, y_train)

SVC()

In [62]:
KNN_pred=KNN_base.predict(x_test)
SVM_pred=SVM_base.predict(x_test)

array=np.column_stack((KNN_pred, SVM_pred))
meta_test=pd.DataFrame(array,columns=['KNN','SVM'])
meta_test.head()

KNN  SVM
0    0    0
1    0    0
2    0    0
3    1    0
4    0    0

In [63]:
print(x_test.shape)
meta_test.shape # okay

(154, 8)


(154, 2)

In [64]:
# Now we have testing data for the meta model. let's test it

meta_pred=meta_learner.predict(meta_test)

In [65]:
meta_pred.shape

(154,)

In [66]:
# Evaluation metric for meta model

from sklearn.metrics import accuracy_score
accuracy_score(y_test, meta_pred)

0.7662337662337663

In [67]:
# accuracy for base KNN and base SVM

print('KNN : ',accuracy_score(y_test, KNN_pred))
print('SVM : ',accuracy_score(y_test, SVM_pred))

KNN :  0.6623376623376623
SVM :  0.7662337662337663


# **Below is the implementation using KNN, SVM, DT as base models**

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('diabetes.csv')
df.head()

# Split features and target
features = df.drop('Outcome', axis=1)
target = df['Outcome']

# Split dataset into training and testing data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize base models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

KNN_base = KNeighborsClassifier()
SVM_base = SVC()
DT_base = DecisionTreeClassifier(random_state=42)

# Cross-validation setup (5-fold)
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

KNN_oof_preds = np.zeros(len(x_train))
SVM_oof_preds = np.zeros(len(x_train))
DT_oof_preds = np.zeros(len(x_train))

# Training base models and getting out-of-fold predictions
for train_index, val_index in kf.split(x_train):
    x_tr, x_val = x_train.iloc[train_index], x_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Fit models
    KNN_base.fit(x_tr, y_tr)
    SVM_base.fit(x_tr, y_tr)
    DT_base.fit(x_tr, y_tr)

    # Collect out-of-fold predictions for validation set
    KNN_oof_preds[val_index] = KNN_base.predict(x_val)
    SVM_oof_preds[val_index] = SVM_base.predict(x_val)
    DT_oof_preds[val_index] = DT_base.predict(x_val)



# Stack out-of-fold predictions for meta-learner input
arr = np.column_stack((KNN_oof_preds, SVM_oof_preds, DT_oof_preds))
print("Shape of stacked out-of-fold predictions:", arr.shape)

meta_train = pd.DataFrame(arr, columns=['KNN', 'SVM','DT'])
meta_train.head()

# Train the meta-learner (Random Forest)
from sklearn.ensemble import RandomForestClassifier
meta_learner = RandomForestClassifier()
meta_learner.fit(meta_train, y_train)

# Make predictions with base models on the test set
KNN_base.fit(x_train, y_train)
SVM_base.fit(x_train, y_train)
DT_base.fit(x_train, y_train)

KNN_pred = KNN_base.predict(x_test)
SVM_pred = SVM_base.predict(x_test)
DT_pred = DT_base.predict(x_test)


# Stack predictions for meta-learner input
array = np.column_stack((KNN_pred, SVM_pred,DT_pred))
meta_test = pd.DataFrame(array, columns=['KNN', 'SVM','DT'])
meta_test.head()

# Evaluate the meta-learner's performance
meta_pred = meta_learner.predict(meta_test)

# Print accuracy scores
from sklearn.metrics import accuracy_score

# Meta-learner accuracy
meta_accuracy = accuracy_score(y_test, meta_pred)
print("Meta Learner Accuracy: ", meta_accuracy)

# Accuracy for base KNN and SVM
print('KNN Accuracy: ', accuracy_score(y_test, KNN_pred))
print('SVM Accuracy: ', accuracy_score(y_test, SVM_pred))
print('DT Accuracy: ', accuracy_score(y_test, DT_pred))


Shape of stacked out-of-fold predictions: (614, 3)
Meta Learner Accuracy:  0.7142857142857143
KNN Accuracy:  0.6623376623376623
SVM Accuracy:  0.7662337662337663
DT Accuracy:  0.7467532467532467
